In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.1 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [6]:
import typing_extensions as typing 

class MedicalHelp(typing.TypedDict): 
    zipcode: int 
    medical_reason: str

In [7]:
client = genai.Client(api_key=GOOGLE_API_KEY)

config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

response = client.models.generate_content(
    model='gemini-2.0-flash', 
    contents="Can you get me the medical centers near zipcode 77030 that can treat broken arm?", 
    config=config_with_search,
)

rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

Here are some medical centers near the 77030-zip code that can treat a broken arm. This information is based on the search results as of April 5, 2025.

**Hospitals and Medical Centers:**

*   **Texas Orthopedic Hospital:** Located at 7401 Main St., Houston, TX 77030. Phone: (713) 799 - 8600. They treat orthopedic trauma and fractures.
*   **HCA Houston Healthcare Medical Center:** Located at 1313 Hermann Dr., Houston, TX 77004. Phone: (713) 527 - 5000. They offer comprehensive orthopedic care, including treatments for broken bones.
*   **Ascent Emergency Medical Center:** This is a 24-hour emergency room located in the 77030-zip code area. They can handle emergencies like broken bones.
*   **Baylor St. Luke's Medical Center - McNair Campus:** Located at 7200 Cambridge St., Houston, TX 77030. Phone: (713) 986-6016. They offer treatments for broken arms and a variety of orthopedic conditions.
*   **Michael E. DeBakey VA Medical Center:** Located at 2002 Holcombe Boulevard, Houston, TX 77030-4211. Main phone: 713-791-1414. They offer primary care and specialty health services, including treatment for spinal cord injuries, and Virtual Urgent Care for non-life-threatening conditions such as sprains, back pain, and joint pain/stiffness..
*   **Children's Memorial Hermann Hospital:** Located at 6411 Fannin, Houston, TX 77030. Phone: (713) 486-4880. They have a Fracture Program that provides specialized care for children's fractures.

**Urgent Care Centers and Orthopedic Clinics:**

*   **Houston Methodist Orthopedics & Sports Medicine:** They have multiple locations across the Greater Houston area. They offer same-day Orthopedic Injury Clinics.
*   **AOKC Urgent OrthoCare:** This clinic provides orthopedic care, sports medicine, and treatments for broken bones and sprains.
*   **Memorial Primary and Urgent Care:** Located at 14629 Memorial Drive, Houston, TX 77079. They offer fracture treatment and can perform diagnostic tests like X-rays.
*   **Barrett S. Brown, MD:** Provides treatments for orthopedic concerns such as a broken proximal humerus. Phone: (346) 247-2531.

When dealing with a broken arm, it's important to seek medical attention promptly. Depending on the severity of the break, you may need to go to an emergency room, urgent care center, or an orthopedic specialist.


In [8]:
from pprint import pprint 

supports = rc.grounding_metadata.grounding_supports
for support in supports: 
    pprint(support.to_json_dict())

{'confidence_scores': [0.98075587, 0.97301334, 0.9792986],
 'grounding_chunk_indices': [0, 1, 2],
 'segment': {'end_index': 268,
             'start_index': 189,
             'text': '*   **Texas Orthopedic Hospital:** Located at 7401 Main '
                     'St., Houston, TX 77030.'}}
{'confidence_scores': [0.7494159, 0.75919265],
 'grounding_chunk_indices': [1, 0],
 'segment': {'end_index': 293,
             'start_index': 269,
             'text': 'Phone: (713) 799 - 8600.'}}
{'confidence_scores': [0.7551733,
                       0.70448554,
                       0.64827263,
                       0.7792259,
                       0.6515472,
                       0.83221406,
                       0.71790147,
                       0.8652005],
 'grounding_chunk_indices': [1, 3, 2, 4, 0, 5, 6, 7],
 'segment': {'end_index': 337,
             'start_index': 294,
             'text': 'They treat orthopedic trauma and fractures.'}}
{'confidence_scores': [0.9710212, 0.93917346],
 